This notebook demonstrates a dataloader that can be used with all three of the machine learning types below.

|<center>name|<center>server|<center>clients|<center>training|<center>training data|<center>evaluation type|<center>evaluation|<center>evaluation data|
|---|---|---|---|---|---|---|---|
|<center>classic machine learning|<center>yes|<center>no|<center>on server|<center>on server|<center>centralized|<center>on server|<center>on server|
|<center>federated machine learning (Central Eval)|<center>yes|<center>yes|<center>on clients|<center>on clients|<center>centralized|<center>on server|<center>on server|
|<center>federated machine learning (Federated Eval)|<center>yes|<center>yes|<center>on clients|<center>on clients|<center>distributed|<center>on clients|<center>on clients|


In [24]:
#load libraries
import math
import os
import glob
import gc
import typing
from typing import List
from typing import Tuple
from typing import Dict
from typing import Optional
import random
import tempfile

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import Model

import flwr as fl
from flwr.common import Metrics

#from tensorflow.keras.models import Model

In [2]:
#overall environment settings

# Make TensorFlow logs less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Training on GPU or CPU?
tf.config.set_visible_devices([], 'GPU')
print(
    f"Training on {'GPU' if tf.config.get_visible_devices('GPU') else 'CPU'} using TensorFlow {tf.__version__} and Flower {fl.__version__}"
)

Training on CPU using TensorFlow 2.12.0 and Flower 1.4.0


In [3]:
#global variables

ml_type = 0 # classic ML = 0, federated ML w/ centralized evaluation = 1, federated ML w/ federated eval = 2

federated_path = "../data/24_clients/"
centralized_path = "../data/01_clients/"

downsample_test_set = 0 # 0 if test set not downsampled, 1 otherwise

# TODO: Add the below to a client config file
sequence_len = 120 # 5 days * 24 hours
past_len = sequence_len
future_len = 24 # 1*24 hours
sampling_rate = 1 # in time series conversion use every row (hour) in loaded datasets
sequence_stride = 1 # in time series conversion each series is this far apart

## ***Data loading functions***

In [4]:
def normalize_data(x):
    """Normalizes the data of an array by column. 
    Shifts and scales inputs into a distribution centered around 0 
    with standard deviation 1.

    Parameters
    ----------
    x: NDarray 
        An array of feature values.
        
    Returns
    -------
    features_normalized : NDarray
        The original array, but normalized.
    """
    data = x
    layer = layers.Normalization()
    layer.adapt(data)
    features_normalized = layer(data)
    return features_normalized

In [5]:
def mask_create(x):
    """Finds the class count of the input array and creates a mask that can be used
    to randomly downsample an array of labels so that the number of 
    negative labels = the number of positive labels. 

    Parameters
    ----------
    x: NDarray 
        An array of feature values.
        
    Returns
    -------
    features_normalized : NDarray
        A masked version of the input array.
    """
    mask_length = x.shape[0]
    mask = tf.reshape(x, [mask_length])
    y, idx, class_count = tf.unique_with_counts(mask)
    ignition_count = tf.get_static_value(class_count[1])
    mask = mask.numpy()
    count = 0 
    while count < ignition_count:
        rand_num = random.randint(0,mask_length)
        if (mask[rand_num] == 0):
            mask[rand_num] = 1
            count += 1
    return mask

In [6]:
def load_datasets(path: str):
    """Loads all the csv datasets in a folder.
    The loaded data is divided into train, validation, and test sets.
    The data is turned into time series data
    All the data is normalized.
    Train and validation datasets are downsampled.
    TODO: divide this function into smaller functions

    Parameters
    ----------
    path: string 
        The path to the dataset folder.
        
    Returns
    -------
    train_x, train_y, val_x, val_y, test_x, test_y : NDarrays
        A masked version of the input array.
    """

    train_x = []
    train_y = []
    val_x = []
    val_y = []
    test_x = []
    test_y = []

    #load data
    for filename in glob.glob(os.path.join(path, '*.csv')):
        print("\nnow reading " + filename + "\n")
        #read file
        df = pd.read_csv(filename, index_col=[0])
        
        df_train = df[(df['year'] < 2001)]
        df_val = df[(df['year'] > 2001) & (df['year'] < 2012)]
        df_test = df[(df['year'] >= 2012)]
        
        features = ['stl2', 't2m', 'stl1', 'stl3', 'skt', 'swvl1', 'd2m', 'swvl2']
        train_features = df_train[features]
        train_labels = df_train[["ignition"]]
        val_features = df_val[features]
        val_labels = df_val[["ignition"]]
        test_features = df_test[features]
        test_labels = df_test[["ignition"]]
        #convert to numpy
        train_features = train_features.values
        val_features = val_features.values
        test_features = test_features.values

        #normalize
        train_features_normalize = normalize_data(train_features)
        val_features_normalize = normalize_data(val_features)
        test_features_normalize = normalize_data(test_features)
        
        #we want to predict at a future point
        #so we clip the length of the features plus the hours till the future point
        start = past_len + future_len
        train_labels = train_labels.iloc[start:].values
        val_labels = val_labels.iloc[start:].values
        test_labels = test_labels.iloc[start:].values
        
        batch_size = 107856 #factor of 5136 (321 * 16)
        #convert to time series data
        train_dataset = keras.utils.timeseries_dataset_from_array(
            train_features_normalize,
            train_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)

        val_dataset = keras.utils.timeseries_dataset_from_array(
            val_features_normalize,
            val_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)

        test_dataset = keras.utils.timeseries_dataset_from_array(
            test_features_normalize,
            test_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)
        
        #for bookkeeping print out the shapes of the datasets
        for train_features, train_labels in train_dataset:
            print("train_dataset features shape:", train_features.shape)
            print("targets_dataset labels shape:", train_labels.shape)
            break

        for val_features, val_labels in val_dataset:
            print("\nval_dataset features shape:", val_features.shape)
            print("val_dataset labels shape:", val_labels.shape)
            break

        for test_features, test_labels in test_dataset:
            print("\ntest_dataset features shape:", test_features.shape)
            print("test_dataset labels shape:", test_labels.shape)
            break
       
        # randomly downsample the data using masks
        train_mask = mask_create(train_labels)
        train_features_masked = tf.boolean_mask(train_features, train_mask)
        train_labels_masked = tf.boolean_mask(train_labels, train_mask)
        
        val_mask = mask_create(val_labels)
        val_features_masked = tf.boolean_mask(val_features, val_mask)
        val_labels_masked = tf.boolean_mask(val_labels, val_mask)
        
        test_mask = mask_create(test_labels)
        test_features_masked = tf.boolean_mask(test_features, test_mask)
        test_labels_masked = tf.boolean_mask(test_labels, test_mask)
        
        train_x.append(train_features_masked)
        train_y.append(train_labels_masked)
        val_x.append(val_features_masked)
        val_y.append(val_labels_masked)
        if (downsample_test_set == 1):
            test_x.append(test_features_masked)
            test_y.append(test_labels_masked)
        else:
            test_x.append(test_features)
            test_y.append(test_labels)
        
    print("\nDone loading data.\n")
    return train_x, train_y, val_x, val_y, test_x, test_y 



In [7]:
def get_value_count(x):
    """A helper function that returns the count of class labels.

    Parameters
    ----------
    x: NDArray 
        An array with class labels.
        
    Returns
    -------
    non_ignition_count, ignition_count : int
        The counts of the ignition class.
    """
    length = x[0].shape[0]
    x = tf.reshape(x, [length])
    y, idx, class_count = tf.unique_with_counts(x)
    non_ignition_count = tf.get_static_value(class_count[0])
    ignition_count = tf.get_static_value(class_count[1])
    return non_ignition_count, ignition_count
    

## ***Model and Metrics***

In [8]:
METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'), 
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    #keras.metrics.F1Score(name='f1_score'),#only available with nightly build
]

def make_model(metrics=METRICS):
    inputs = keras.Input(shape=(sequence_len, trainloaders_x[0].shape[2]))
    x = layers.LSTM(8, activation='sigmoid')(inputs)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics)
        
    return model

#model = make_model()

## ***Federated Learning Functions***

In [9]:
# TODO: add epochs to client config file 
epochs = 10

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_val, y_val, tb_callback) -> None:
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val
        self.tb_callback = tb_callback

    def get_parameters(self, config):
        return self.model.get_weights()

    # presetting config allows us to use FlowerClient with classic ml.
    def fit(self, parameters, config = "ServerConfig(num_rounds=5, round_timeout=None)"):
        print("in fit")
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=epochs, verbose=2, callbacks=self.tb_callback)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config = "ServerConfig(num_rounds=5, round_timeout=None)"):
        self.model.set_weights(parameters)
        loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc  = self.model.evaluate(self.x_val, self.y_val, verbose=2)
        return loss, len(self.x_val), {"tp": tp,
                                       "fp": fp,
                                       "tn": tn,
                                       "fn": fn,
                                       "accuracy": accuracy,
                                       "precision": precision,
                                       "recall": recall,
                                       "auc": auc,
                                       "prc": prc
                                      }
    
    # this method allows metrics to be passed when using a single server with no clients
    def evaluate2(self, parameters, config = "ServerConfig(num_rounds=5, round_timeout=None)"):
        self.model.set_weights(parameters)
        loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc = self.model.evaluate(self.x_val, self.y_val, verbose=2)
        return loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc

In [10]:
def client_fn(cid: str) -> fl.client.Client:

    print("\nThis is client: ", cid)

    x_train_cid = trainloaders_x[int(cid)]
    y_train_cid = trainloaders_y[int(cid)]
    x_test_cid = testloaders_x[int(cid)]
    y_test_cid = testloaders_y[int(cid)]

    print("Loaded data for client: ", cid, "\n")

    METRICS = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
        #keras.metrics.F1Score(name='f1_score'),#only available with nightly build
    ]

    def make_model(metrics=METRICS):
        inputs = keras.Input(shape=(sequence_len, x_train_cid.shape[2]))
        x = layers.LSTM(8, activation='sigmoid')(inputs)
        x = layers.Flatten()(x)
        outputs = layers.Dense(1, activation="sigmoid")(x)
        model = keras.Model(inputs, outputs)

        model.compile(
            optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-3),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=metrics)
        
        return model
    
    print("Making model: ", cid)
    model = make_model()

    #model.summary()
    import datetime
    log_dir = "./logs/fit/" + cid + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    tb_callback = keras.callbacks.TensorBoard(log_dir="logs/", histogram_freq=1)

    # Create and return client
    print("\nClient CID: " + str(cid) + " is done.\n")
    return FlowerClient(model, x_train_cid, y_train_cid, x_test_cid, y_test_cid, tensorboard)

## ***Launch classic machine learning***

In [11]:
#TODO: there is a random out-of-bounds error with masking, for now if it occurs run this cell again
#load the dataset for centralized evaluation (for classic ml training and testing) 
trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y = load_datasets(centralized_path)


now reading ../data/01_clients/dly_avg_1of1_50.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

Done loading data.



In [12]:
#save copy for federated centralized evaluation
central_testloaders_x = testloaders_x.copy()
central_testloaders_y = testloaders_y.copy()

In [14]:
count = get_value_count(trainloaders_y)
print("Train set nonignitions and ignitions are:", count)
count = get_value_count(valloaders_y)
print("Validation set nonignitions and ignitions are:", count)
count = get_value_count(testloaders_y)
print("Test set nonignitions and ignitions are:", count)
count = get_value_count(central_testloaders_y)
print("Central test set nonignitions and ignitions are:", count)

Train set nonignitions and ignitions are: (2811, 2811)
Validation set nonignitions and ignitions are: (2534, 2534)
Test set nonignitions and ignitions are: (44719, 1361)
Central test set nonignitions and ignitions are: (44719, 1361)


In [15]:
model = make_model()

In [16]:
#train single server classic ml
#create a flower client that represents a classic ml single server
classic_ml = client_fn(str(0))
#set parameters for classic_ml so it can use single flower client function
parameters = model.get_weights()
#run fit
history = classic_ml.fit(parameters)


This is client:  0
Loaded data for client:  0 

Making model:  0

Client CID: 0 is done.

in fit
Epoch 1/10
176/176 - 2s - loss: 0.6299 - tp: 2136.0000 - fp: 1319.0000 - tn: 1492.0000 - fn: 675.0000 - accuracy: 0.6453 - precision: 0.6182 - recall: 0.7599 - auc: 0.7183 - prc: 0.7218 - 2s/epoch - 12ms/step
Epoch 2/10
176/176 - 1s - loss: 0.6127 - tp: 1887.0000 - fp: 922.0000 - tn: 1889.0000 - fn: 924.0000 - accuracy: 0.6716 - precision: 0.6718 - recall: 0.6713 - auc: 0.7262 - prc: 0.7321 - 1s/epoch - 7ms/step
Epoch 3/10
176/176 - 1s - loss: 0.6078 - tp: 1870.0000 - fp: 881.0000 - tn: 1930.0000 - fn: 941.0000 - accuracy: 0.6759 - precision: 0.6798 - recall: 0.6652 - auc: 0.7305 - prc: 0.7390 - 1s/epoch - 7ms/step
Epoch 4/10
176/176 - 1s - loss: 0.6029 - tp: 1849.0000 - fp: 827.0000 - tn: 1984.0000 - fn: 962.0000 - accuracy: 0.6818 - precision: 0.6910 - recall: 0.6578 - auc: 0.7354 - prc: 0.7428 - 1s/epoch - 7ms/step
Epoch 5/10
176/176 - 1s - loss: 0.5982 - tp: 1857.0000 - fp: 805.0000 - 

In [17]:
#set parameters for classic_ml so it can use single flower client function
parameters = model.get_weights()

In [18]:
#evaluate single server classic ml
loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc = classic_ml.evaluate2(parameters)

1440/1440 - 3s - loss: 0.8344 - tp: 1296.0000 - fp: 35672.0000 - tn: 9047.0000 - fn: 65.0000 - accuracy: 0.2245 - precision: 0.0351 - recall: 0.9522 - auc: 0.7307 - prc: 0.0662 - 3s/epoch - 2ms/step


In [19]:
print("Single server classic ml evaluation\n\
\ttp:\t%d\n\
\tfp:\t%d\n\
\ttn:\t%d\n\
\tfn:\t%d\n\n\
\tloss:\t%f\n\
\tacc:\t%f\n\
\tprec:\t%f\n\
\trec:\t%f\n\
\tauc:\t%f\n\
\tprc:\t%f\
" % (tp,fp,tn,fn,loss,accuracy,precision,recall,auc,prc))

Single server classic ml evaluation
	tp:	1296
	fp:	35672
	tn:	9047
	fn:	65

	loss:	0.834442
	acc:	0.224457
	prec:	0.035057
	rec:	0.952241
	auc:	0.730658
	prc:	0.066188


## ***Federated machine learning***

In [21]:
#TODO: there is a random out-of-bounds error with masking, for now if it occurs run this cell again
#load the dataset for federated learning
trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y = load_datasets(federated_path)


now reading ../data/24_clients/121.625_50.25_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

now reading ../data/24_clients/119.625_51.75_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

now reading ../data/24_clients/121.625_49.5_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

now reading ../data/24_clients/116.625_51_cell.csv

train_dataset f

In [22]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    tps = [num_examples * m["tp"] for num_examples, m in metrics]
    fps = [num_examples * m["fp"] for num_examples, m in metrics]
    tns = [num_examples * m["tn"] for num_examples, m in metrics]
    fns = [num_examples * m["fn"] for num_examples, m in metrics]
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    precisions = [num_examples * m["precision"] for num_examples, m in metrics]
    recalls = [num_examples * m["recall"] for num_examples, m in metrics]
    aucs = [num_examples * m["auc"] for num_examples, m in metrics]
    prcs = [num_examples * m["prc"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"tp": sum(tps) / sum(examples),
            "fp": sum(fps) / sum(examples),
            "tn": sum(tns) / sum(examples),
            "fn": sum(fns) / sum(examples),
            "accuracy": sum(accuracies) / sum(examples),
            "precision": sum(precisions) / sum(examples),
            "recall": sum(recalls) / sum(examples),
            "auc": sum(aucs) / sum(examples),
            "prc": sum(prcs) / sum(examples)
           }

In [21]:
NUM_CLIENTS = 2
epochs = 2
num_rounds = 2
# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Sample 10% of available clients for training
    fraction_evaluate=1.0,  # Sample 5% of available clients for evaluation
    min_fit_clients=1,  # Never sample less than 10 clients for training
    min_evaluate_clients=1,  # Never sample less than 5 clients for evaluation
    min_available_clients=int(NUM_CLIENTS * 0.5),  # Wait until at least 75 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=num_rounds),
    strategy=strategy,
)

INFO flwr 2023-06-04 07:31:08,009 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=2, round_timeout=None)
2023-06-04 07:31:09,759	INFO worker.py:1625 -- Started a local Ray instance.
INFO flwr 2023-06-04 07:31:10,358 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 77318471680.0, 'CPU': 12.0, 'object_store_memory': 2147483648.0, 'node:127.0.0.1': 1.0}
INFO flwr 2023-06-04 07:31:10,359 | server.py:86 | Initializing global parameters
INFO flwr 2023-06-04 07:31:10,359 | server.py:273 | Requesting initial parameters from one random client


(launch_and_get_parameters pid=26814) Metal device set to: Apple M2 Max


(launch_and_get_parameters pid=26814) WARNING:tensorflow:Layer lstm will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.
INFO flwr 2023-06-04 07:31:15,616 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-06-04 07:31:15,617 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-06-04 07:31:15,617 | server.py:101 | FL starting
DEBUG flwr 2023-06-04 07:31:15,617 | server.py:218 | fit_round 1: strategy sampled 2 clients (out of 2)


(launch_and_get_parameters pid=26814) 
(launch_and_get_parameters pid=26814) This is client:  1
(launch_and_get_parameters pid=26814) Loaded data for client:  1 
(launch_and_get_parameters pid=26814) 
(launch_and_get_parameters pid=26814) Making model:  1
(launch_and_get_parameters pid=26814) 
(launch_and_get_parameters pid=26814) Client CID: 1 is done.
(launch_and_get_parameters pid=26814) 


(launch_and_fit pid=26814) WARNING:tensorflow:Layer lstm_1 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


...

(launch_and_evaluate pid=26814) WARNING:tensorflow:Layer lstm_4 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.
(launch_and_evaluate pid=26816) WARNING:tensorflow:Layer lstm_3 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


(launch_and_evaluate pid=26814) 
(launch_and_evaluate pid=26814) This is client:  0
(launch_and_evaluate pid=26814) Loaded data for client:  0 
(launch_and_evaluate pid=26814) 
(launch_and_evaluate pid=26814) Making model:  0
(launch_and_evaluate pid=26814) Client CID: 0 is done.
(launch_and_evaluate pid=26816) 1440/1440 - 816s - loss: 0.6533 - tp: 171.0000 - fp: 17501.0000 - tn: 28382.0000 - fn: 26.0000 - accuracy: 0.6196 - precision: 0.0097 - recall: 0.8680 - auc: 0.8206 - prc: 0.0183 - 816s/epoch - 567ms/step
(launch_and_evaluate pid=26816)  [repeated 6x across cluster]
(launch_and_evaluate pid=26816) 1440/1440 - 816s - loss: 0.6533 - tp: 171.0000 - fp: 17501.0000 - tn: 28382.0000 - fn: 26.0000 - accuracy: 0.6196 - precision: 0.0097 - recall: 0.8680 - auc: 0.8206 - prc: 0.0183 - 816s/epoch - 567ms/step
(launch_and_evaluate pid=26816) 1440/1440 - 816s - loss: 0.6533 - tp: 171.0000 - fp: 17501.0000 - tn: 28382.0000 - fn: 26.0000 - accuracy: 0.6196 - precision: 0.0097 - recall: 0.8680 

DEBUG flwr 2023-06-04 08:41:20,253 | server.py:182 | evaluate_round 2 received 2 results and 0 failures
INFO flwr 2023-06-04 08:41:20,253 | server.py:147 | FL finished in 4204.607913084001
INFO flwr 2023-06-04 08:41:20,254 | app.py:218 | app_fit: losses_distributed [(1, 0.7046457231044769), (2, 0.654981255531311)]
INFO flwr 2023-06-04 08:41:20,254 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-06-04 08:41:20,255 | app.py:220 | app_fit: metrics_distributed {'tp': [(1, 216.5), (2, 202.0)], 'fp': [(1, 21308.0), (2, 17482.5)], 'tn': [(1, 24489.0), (2, 28314.5)], 'fn': [(1, 66.5), (2, 81.0)], 'accuracy': [(1, 0.5361436605453491), (2, 0.618847668170929)], 'precision': [(1, 0.010062229819595814), (2, 0.011421198956668377)], 'recall': [(1, 0.7961220443248749), (2, 0.7497282922267914)], 'auc': [(1, 0.7337964773178101), (2, 0.7310915589332581)], 'prc': [(1, 0.01630517654120922), (2, 0.015765988267958164)]}
INFO flwr 2023-06-04 08:41:20,255 | app.py:221 | app_fit: losses_centra

History (loss, distributed):
	round 1: 0.7046457231044769
	round 2: 0.654981255531311
History (metrics, distributed, evaluate):
{'tp': [(1, 216.5), (2, 202.0)], 'fp': [(1, 21308.0), (2, 17482.5)], 'tn': [(1, 24489.0), (2, 28314.5)], 'fn': [(1, 66.5), (2, 81.0)], 'accuracy': [(1, 0.5361436605453491), (2, 0.618847668170929)], 'precision': [(1, 0.010062229819595814), (2, 0.011421198956668377)], 'recall': [(1, 0.7961220443248749), (2, 0.7497282922267914)], 'auc': [(1, 0.7337964773178101), (2, 0.7310915589332581)], 'prc': [(1, 0.01630517654120922), (2, 0.015765988267958164)]}

(launch_and_evaluate pid=26814) 1440/1440 - 826s - loss: 0.6567 - tp: 233.0000 - fp: 17464.0000 - tn: 28247.0000 - fn: 136.0000 - accuracy: 0.6181 - precision: 0.0132 - recall: 0.6314 - auc: 0.6416 - prc: 0.0133 - 826s/epoch - 573ms/step


In [32]:
NUM_CLIENTS = 2
epochs = 2
num_rounds = 2

client_resources = {"num_cpus": 2}
if tf.config.get_visible_devices("GPU"):
    client_resources["num_gpus"] = 1

def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    """Centralized evaluation function"""
    model = make_model()
    #model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
    #model.build(input_shape=(BATCH_SIZE, 28, 28, 1))
    model.set_weights(parameters)
#    loss, accuracy = model.evaluate(central_testloaders_x, central_testloaders_y, batch_size=32, verbose=0)
#    return loss, {"accuracy": accuracy}
    loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc  = model.evaluate(central_testloaders_x[0], central_testloaders_y[0], verbose=2)
    return loss, {"tp": tp,
                  "fp": fp,
                  "tn": tn,
                  "fn": fn,
                  "accuracy": accuracy,
                  "precision": precision,
                  "recall": recall,
                  "auc": auc,
                  "prc": prc
                 }

# TODO: Specify the Strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Sample 10% of available clients for training
    fraction_evaluate=1.0,  # Sample 5% of available clients for evaluation
    min_fit_clients=1,  # Never sample less than 10 clients for training
    min_evaluate_clients=1,  # Never sample less than 5 clients for evaluation
    min_available_clients=int(NUM_CLIENTS * 0.5),  # Wait until at least 75 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,
    evaluate_fn=evaluate
)

# Start simulation
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=num_rounds),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-06-04 10:42:32,634 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=2, round_timeout=None)
2023-06-04 10:42:35,986	INFO worker.py:1625 -- Started a local Ray instance.
INFO flwr 2023-06-04 10:42:36,527 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 12.0, 'memory': 76328850228.0, 'object_store_memory': 2147483648.0, 'node:127.0.0.1': 1.0}
INFO flwr 2023-06-04 10:42:36,527 | server.py:86 | Initializing global parameters
INFO flwr 2023-06-04 10:42:36,527 | server.py:273 | Requesting initial parameters from one random client


(launch_and_get_parameters pid=29924) Metal device set to: Apple M2 Max


(launch_and_get_parameters pid=29924) WARNING:tensorflow:Layer lstm will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.
INFO flwr 2023-06-04 10:42:41,661 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-06-04 10:42:41,661 | server.py:88 | Evaluating initial parameters


(launch_and_get_parameters pid=29924) 
(launch_and_get_parameters pid=29924) This is client:  1
(launch_and_get_parameters pid=29924) Loaded data for client:  1 
(launch_and_get_parameters pid=29924) 
(launch_and_get_parameters pid=29924) Making model:  1
(launch_and_get_parameters pid=29924) 
(launch_and_get_parameters pid=29924) Client CID: 1 is done.
(launch_and_get_parameters pid=29924) 
1440/1440 - 3s - loss: 0.4413 - tp: 1313.0000 - fp: 39568.0000 - tn: 94589.0000 - fn: 2770.0000 - accuracy: 0.6937 - precision: 0.0321 - recall: 0.3216 - auc: 0.5397 - prc: 0.0390 - 3s/epoch - 2ms/step


INFO flwr 2023-06-04 10:42:44,459 | server.py:91 | initial parameters (loss, other metrics): 0.4412809908390045, {'tp': 1313.0, 'fp': 39568.0, 'tn': 94589.0, 'fn': 2770.0, 'accuracy': 0.6937355399131775, 'precision': 0.032117608934640884, 'recall': 0.3215772807598114, 'auc': 0.5396873950958252, 'prc': 0.03895753622055054}
INFO flwr 2023-06-04 10:42:44,459 | server.py:101 | FL starting
DEBUG flwr 2023-06-04 10:42:44,460 | server.py:218 | fit_round 1: strategy sampled 2 clients (out of 2)


...

DEBUG flwr 2023-06-04 11:38:52,335 | server.py:232 | fit_round 2 received 2 results and 0 failures


(launch_and_fit pid=29926) 53/53 - 620s - loss: 0.6639 - tp: 506.0000 - fp: 317.0000 - tn: 522.0000 - fn: 333.0000 - accuracy: 0.6126 - precision: 0.6148 - recall: 0.6031 - auc: 0.6426 - prc: 0.6287 - 620s/epoch - 12s/step
1440/1440 - 3s - loss: 0.6068 - tp: 3094.0000 - fp: 67255.0000 - tn: 156340.0000 - fn: 3711.0000 - accuracy: 0.6920 - precision: 0.0440 - recall: 0.4547 - auc: 0.6010 - prc: 0.0463 - 3s/epoch - 2ms/step


INFO flwr 2023-06-04 11:38:55,069 | server.py:119 | fit progress: (2, 0.6067822575569153, {'tp': 3094.0, 'fp': 67255.0, 'tn': 156340.0, 'fn': 3711.0, 'accuracy': 0.6919878721237183, 'precision': 0.04398072510957718, 'recall': 0.4546656906604767, 'auc': 0.601047694683075, 'prc': 0.04631812870502472}, 3370.5697214580005)
DEBUG flwr 2023-06-04 11:38:55,069 | server.py:168 | evaluate_round 2: strategy sampled 2 clients (out of 2)
(launch_and_evaluate pid=29924) WARNING:tensorflow:Layer lstm_4 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.
(launch_and_evaluate pid=29926) WARNING:tensorflow:Layer lstm_3 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


(launch_and_evaluate pid=29924) 
(launch_and_evaluate pid=29924) This is client:  0
(launch_and_evaluate pid=29924) Loaded data for client:  0 
(launch_and_evaluate pid=29924) 
(launch_and_evaluate pid=29924) Making model:  0
(launch_and_evaluate pid=29924) 
(launch_and_evaluate pid=29924) Client CID: 0 is done.
(launch_and_evaluate pid=29924) 
(launch_and_evaluate pid=29924) 1440/1440 - 761s - loss: 0.6143 - tp: 230.0000 - fp: 16944.0000 - tn: 28767.0000 - fn: 139.0000 - accuracy: 0.6293 - precision: 0.0134 - recall: 0.6233 - auc: 0.6351 - prc: 0.0123 - 761s/epoch - 529ms/step
(launch_and_evaluate pid=29926)  [repeated 4x across cluster]
(launch_and_evaluate pid=29926) 1440/1440 - 761s - loss: 0.6143 - tp: 230.0000 - fp: 16944.0000 - tn: 28767.0000 - fn: 139.0000 - accuracy: 0.6293 - precision: 0.0134 - recall: 0.6233 - auc: 0.6351 - prc: 0.0123 - 761s/epoch - 529ms/step
(launch_and_evaluate pid=29926) 1440/1440 - 761s - loss: 0.6143 - tp: 230.0000 - fp: 16944.0000 - tn: 28767.0000 - 

DEBUG flwr 2023-06-04 11:51:39,113 | server.py:182 | evaluate_round 2 received 2 results and 0 failures
INFO flwr 2023-06-04 11:51:39,113 | server.py:147 | FL finished in 4134.607445916
INFO flwr 2023-06-04 11:51:39,113 | app.py:218 | app_fit: losses_distributed [(1, 0.5799275934696198), (2, 0.6118367314338684)]
INFO flwr 2023-06-04 11:51:39,114 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-06-04 11:51:39,114 | app.py:220 | app_fit: metrics_distributed {'tp': [(1, 167.5), (2, 200.0)], 'fp': [(1, 12515.5), (2, 16813.5)], 'tn': [(1, 33281.5), (2, 28983.5)], 'fn': [(1, 115.5), (2, 83.0)], 'accuracy': [(1, 0.7258897721767426), (2, 0.6333225071430206)], 'precision': [(1, 0.013150867074728012), (2, 0.011739781126379967)], 'recall': [(1, 0.6242898404598236), (2, 0.7431252002716064)], 'auc': [(1, 0.721286952495575), (2, 0.7235794961452484)], 'prc': [(1, 0.013554776553064585), (2, 0.01395477820187807)]}
INFO flwr 2023-06-04 11:51:39,114 | app.py:221 | app_fit: losses_central

In [33]:
history

History (loss, distributed):
	round 1: 0.5799275934696198
	round 2: 0.6118367314338684
History (loss, centralized):
	round 0: 0.4412809908390045
	round 1: 0.5796513557434082
	round 2: 0.6067822575569153
History (metrics, distributed, evaluate):
{'tp': [(1, 167.5), (2, 200.0)], 'fp': [(1, 12515.5), (2, 16813.5)], 'tn': [(1, 33281.5), (2, 28983.5)], 'fn': [(1, 115.5), (2, 83.0)], 'accuracy': [(1, 0.7258897721767426), (2, 0.6333225071430206)], 'precision': [(1, 0.013150867074728012), (2, 0.011739781126379967)], 'recall': [(1, 0.6242898404598236), (2, 0.7431252002716064)], 'auc': [(1, 0.721286952495575), (2, 0.7235794961452484)], 'prc': [(1, 0.013554776553064585), (2, 0.01395477820187807)]}History (metrics, centralized):
{'tp': [(0, 1313.0), (1, 2128.0), (2, 3094.0)], 'fp': [(0, 39568.0), (1, 51477.0), (2, 67255.0)], 'tn': [(0, 94589.0), (1, 127399.0), (2, 156340.0)], 'fn': [(0, 2770.0), (1, 3316.0), (2, 3711.0)], 'accuracy': [(0, 0.6937355399131775), (1, 0.7027289271354675), (2, 0.6919878